# Лекция 16: диспетчеризация урлов, представления и шаблонизация верстки в Django

## Немного принципов
* KISS
* DRY
* NIH
* YAGNI

## Диспетчеризация урлов

### ЧПУ

ЧПУ - (человеко-понятные урлы) - идея формирования "хороших" урлов, которые были бы удобнее, в первую очередь, для человека, а не машины:  
  * содержат читаемые слова и даты  
  * латинский алфавит  
  * показывают вложенность директорий сайта  
  * отражают название сущности, на которую они ссылаются  
  * избегаются технические части текста (расширения, спецсимволы)

### URLconf

* URL dispatcher - (диспетчеризация урлов, routes, роуты) - способ задания схемы отображения из урла запроса на обработчик запроса.
* URLconf - неформальное название задаваемого разработчиком отображения с помощью встроенного в Django механизма.
* Содержится в файлах urls.py.
* Отображение задаётся с помощью кода на Python, которому указывается простое регулярное выражение (описание урла) и обрабатывающий объект-представление.
* URLConf могут вкладываться один в другой - обычно в корневой URLConfg проекта включаются URLConf дочерних приложений.

Пример:

In [ ]:
from django.conf.urls import url

from . import views

urlpatterns = [
    url(r'^articles/2003/$', views.special_case_2003),
    url(r'^articles/([0-9]{4})/$', views.year_archive, name='news-year-archive'),
    url(r'^articles/([0-9]{4})/([0-9]{2})/$', views.month_archive),
    url(r'^articles/([0-9]{4})/([0-9]{2})/([0-9]+)/$', views.article_detail),
]

Иногда нужно искать url по шаблону (чтобы было проще вносить изменения и меньше повторяться).

Для этого существуют механизмы как в Python коде:

In [ ]:
reverse('news-year-archive', args=(year,))

Так и в шаблонах:

In [ ]:
<a href="{% url 'news-year-archive' 2012 %}">2012 Archive</a>

* Подробнее про возможности URLConf:
  * https://docs.djangoproject.com/en/1.11/topics/http/urls/
  * http://djbook.ru/rel1.9/topics/http/urls.html

## Представления

* Представление - (view, вид) - функция (или класс определённого вида) на языке Python, в Django с помощью их реализуются обработчики запросов.  
* Принимает на вход объект-запрос (*django.http.HttpRequest*) и возвращает объект-ответ (*django.http.HttpResponse*).
* Ответ должен задаваться во view, обычно он генерирует html.

Пример:

In [ ]:
from django.http import HttpResponse
import datetime

def current_datetime(request):
    now = datetime.datetime.now()
    html = "<html><body>It is now %s.</body></html>" % now
    return HttpResponse(html)

In [ ]:
from django.http import HttpResponse, HttpResponseNotFound

def my_view(request):
    # ...
    if foo:
        return HttpResponseNotFound('<h1>Page not found</h1>')
    else:
        return HttpResponse('<h1>Page was found</h1>')

In [ ]:
# default will be used or specify your own 404.html template
from django.http import Http404
from django.shortcuts import render_to_response
from polls.models import Poll

def detail(request, poll_id):
    try:
        p = Poll.objects.get(pk=poll_id)
    except Poll.DoesNotExist:
        raise Http404("Poll does not exist")
    return render_to_response('polls/detail.html', {'poll': p})

### Вспомогательные функции

* Лежат в django.shortcuts.
* Позволяют упростить ряд рутинных или часто нужных действий.

#### render 
Рендерит шаблон и создаёт сооветствующий HttpResponse.

In [ ]:
from django.shortcuts import render

def my_view(request):
    # View code here...
    return render(request, 'myapp/index.html', {"foo": "bar"},
        content_type="application/xhtml+xml")

In [ ]:
from django.http import HttpResponse
from django.template import RequestContext, loader

def my_view(request):
    # View code here...
    t = loader.get_template('myapp/index.html')
    c = RequestContext(request, {'foo': 'bar'})
    return HttpResponse(t.render(c),
        content_type="application/xhtml+xml")

#### redirect

Позволяет выполнить редирект путём возвращения специального ответа, создает HttpResponseRedirect на соответствующий url.

In [ ]:
from django.shortcuts import redirect

def my_view(request):
    ...
    object = MyModel.objects.get(...)
    return redirect(object)

In [ ]:
def my_view(request):
    ...
    return redirect('some-view-name', foo='bar')

In [ ]:
def my_view(request):
    ...
    return redirect('/some/url/')

In [ ]:
def my_view(request):
    ...
    return redirect('http://example.com/')

#### get_object_or_404

Оборачивает получение объекта из базы (get): если не найдет, то вернёт ответ 404.

In [ ]:
from django.shortcuts import get_object_or_404

def my_view(request):
    my_object = get_object_or_404(MyModel, pk=1)

In [ ]:
from django.http import Http404

def my_view(request):
    try:
        my_object = MyModel.objects.get(pk=1)
    except MyModel.DoesNotExist:
        raise Http404("No MyModel matches the given query.")

In [ ]:
queryset = Book.objects.filter(title__startswith='M')
get_object_or_404(queryset, pk=1)

#### get_list_or_404

Как предыдущий, только для нескольких объектов (filter).

In [ ]:
from django.shortcuts import get_list_or_404

def my_view(request):
    my_objects = get_list_or_404(MyModel, published=True)

In [ ]:
from django.http import Http404

def my_view(request):
    my_objects = list(MyModel.objects.filter(published=True))
    if not my_objects:
        raise Http404("No MyModel matches the given query.")

Подробнее про представления:
* https://docs.djangoproject.com/en/1.11/#the-view-layer
* http://djbook.ru/rel1.9/#the-view-layer

## Шаблонизация в Django

### Шаблонизация верстки

* Django динамически генерирует верстку.
* Для этого используется механизм шаблонов.
* Django поддерживает выбор бэкэнда шаблонизации.
* Шаблоны предоставляет API, через которое можно загружать и рендерить шаблоны независимо от конкретного бэкэнда.
  * Загрузка - поиск шаблона и его предобработка.
  * Рендеринг - преобразование шаблона и доп. данных текст (обычно html).
* Можно использовать или встроенный механизм Django шаблонов или какой-нибудь внешний шаблонизатор (например, jinja2).

### Язык шаблонов Django

* Django-шаблон - строка или тестовый файл, содержащий элементы языка django-шаблонов и текст.
* Текст может быть разных форматов: html, json, csv, javascript и т.д.

Основные элементы языка шаблонов Django:
* переменные (и фильтры над ними)
* теги

Пример:

In [ ]:
{% extends "base_generic.html" %}

{% block title %}{{ section.title }}{% endblock %}

{% block content %}
<h1>{{ section.title }}</h1>

{% for story in story_list %}
<h2>
  <a href="{{ story.get_absolute_url }}">
    {{ story.headline|upper }}
  </a>
</h2>
<p>{{ story.tease|truncatewords:"100" }}</p>
{% endfor %}
{% endblock %}

#### Переменные

* Записываются как {{ variable }} и {{ section.title }}.
* Переменная заменяется результатом её вычисления.
* Вычисление переменной:
  * Если простое имя, то берётся соответствующее значение из переданного контекста шаблона.
  * Если составное имя с точками, то последовательно  происходит обращение разыменование частей пути значению.
* Когда в имени переменной содержится точка, то шаблонизатор пытается:
  * получить значение по ключу из словаря (могут быть неожиданности, если хотели метод, например у defaultdict)
  * использовать объект как атрибут или метод (вызывается без аргументов и результат используется как значение).
  * индекс списка

#### Фильтры

Фильтры позволяют динамически выполнять цепочечные преобразования над значением переменной.

Пример применения фильтра lower к переменной name: 

In [ ]:
{{ name|lower }}

Цепочка фильтров:

In [ ]:
{{ text|escape|linebreaks }}

Передача аргумента фильтру:

In [ ]:
{{ text|truncatewords:30 }}

Ещё несколько примеров:

In [ ]:
{{ value|default:"nothing" }}

In [ ]:
{{ value|length }}

In [ ]:
{{ value|filesizeformat }}

Подробнее про встроенные фильтры:
* http://djbook.ru/rel1.9/ref/templates/builtins.html#ref-templates-builtins-filters
* https://docs.djangoproject.com/en/1.11/ref/templates/builtins/#ref-templates-builtins-filters

#### Тэги

Тэг - инструкция языка Django-шаблонов. Есть множество уже существующих, а также можно создавать свои.

Рассмотрим некоторые встроенные:

*for*

In [ ]:
<ul>
{% for athlete in athlete_list %}
    <li>{{ athlete.name }}</li>
{% endfor %}
</ul>

*if, elif, и else*

In [ ]:
{% if athlete_list %}
    Number of athletes: {{ athlete_list|length }}
{% elif athlete_in_locker_room_list %}
    Athletes should be out of the locker room soon!
{% else %}
    No athletes.
{% endif %}

In [ ]:
{% if athlete_list|length > 1 %}
   Team: {% for athlete in athlete_list %} ... {% endfor %}
{% else %}
   Athlete: {{ athlete_list.0.name }}
{% endif %}

Комментарии:

In [ ]:
{# greeting #}hello

In [ ]:
{# {% if foo %}bar{% else %} #}

In [ ]:
<p>Rendered text with {{ pub_date|date:"c" }}</p>
{% comment "Optional note" %}
    <p>Commented out text with {{ create_date|date:"c" }}</p>
{% endcomment %}

Подробнее про встроенные теги:
* http://djbook.ru/rel1.9/ref/templates/builtins.html#ref-templates-builtins-tags
* https://docs.djangoproject.com/en/1.11/ref/templates/builtins/#ref-templates-builtins-tags

#### Наследование шаблонов

* Механизм наследования шаблонов позволяет переиспользовать общие части шаблонов.
* Создаём базовый шаблон (шаблоны).
* Помечаем в этих шаблонах специальным образом блоки.
* В шаблонах-потомках можем переопределять только эти блоки (или их часть), а остальное рендерится так, как у предка.

Базовый шаблон (base.html):

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <link rel="stylesheet" href="style.css" />
    <title>{% block title %}My amazing site{% endblock %}</title>
</head>

<body>
    <div id="sidebar">
        {% block sidebar %}
        <ul>
            <li><a href="/">Home</a></li>
            <li><a href="/blog/">Blog</a></li>
        </ul>
        {% endblock %}
    </div>

    <div id="content">
        {% block content %}{% endblock %}
    </div>
</body>
</html>

Дочерний шаблон:

In [ ]:
{% extends "base.html" %}

{% block title %}My amazing blog{% endblock %}

{% block content %}
{% for entry in blog_entries %}
    <h2>{{ entry.title }}</h2>
    <p>{{ entry.body }}</p>
{% endfor %}
{% endblock %}

Пример результата:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <link rel="stylesheet" href="style.css" />
    <title>My amazing blog</title>
</head>

<body>
    <div id="sidebar">
        <ul>
            <li><a href="/">Home</a></li>
            <li><a href="/blog/">Blog</a></li>
        </ul>
    </div>

    <div id="content">
        <h2>Entry one</h2>
        <p>This is my first entry.</p>

        <h2>Entry two</h2>
        <p>This is my second entry.</p>
    </div>
</body>
</html>

Подробно про язык django-шаблонов:
* http://djbook.ru/rel1.9/ref/templates/language.html
* https://docs.djangoproject.com/en/1.11/ref/templates/language/

### Обсуждение и самостоятельное изучение

* Формы в Django
  * http://djbook.ru/rel1.9/#forms
  * https://docs.djangoproject.com/en/1.11/#forms
* Интерфейс администратора
  * http://djbook.ru/rel1.9/#the-admin
  * https://docs.djangoproject.com/en/1.11/#the-admin
* Процесс разработки
  * http://djbook.ru/rel1.9/#the-development-process
  * https://docs.djangoproject.com/en/1.11/#the-development-process
* Безопасность
  * http://djbook.ru/rel1.9/#security
  * https://docs.djangoproject.com/en/1.11/#security
* Интернационализация и локализация
  * http://djbook.ru/rel1.9/#internationalization-and-localization
  * https://docs.djangoproject.com/en/1.11/#internationalization-and-localization